In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import re

In [2]:
# upload all files
df1 = pd.read_csv('punks_atributes1.csv',index_col='id')
df2 = pd.read_csv('punks_atributes2.csv',index_col='id')
df3 = pd.read_csv('punks_atributes3.csv',index_col='id')
df4 = pd.read_csv('punks_atributes4.csv',index_col='id')
df5 = pd.read_csv('punks_atributes5.csv',index_col='id')

In [3]:
# merge into a single csv
df = pd.concat([df1,df2,df3,df4,df5],ignore_index=True)
df.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
# tenemos 10,000 punks
df.shape

(10000, 5)

In [5]:
df.head(10)

,types,rarity,skin,total_traits,traits
0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow,Hidden Ear..."
1,Male,4352,Dark,2,"Mohawk,Smile"
2,Female,8090,Light,1,Wild Hair
3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair"
4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair"
5,Female,1960,Mid,3,"Earring,Half Shaved,Purple Eye Shadow,Hidden E..."
6,Male,7425,Mid,1,Do-rag
7,Female,866,Mid,3,"Clown Eyes Blue,Spots,Wild White Hair"
8,Male,6530,Dark,2,"Luxurious Beard,Messy Hair"
9,Male,1761,Mid,3,"Big Beard,Clown Nose,Police Cap"


In [6]:
df.tail()

,types,rarity,skin,total_traits,traits
9995,Female,2466,Albino,2,"Purple Eye Shadow,Straight Hair Dark"
9996,Male,4323,Light,4,"Cigarette,Crazy Hair,Earring,Smile"
9997,Zombie,498,Zombie,2,"Cap Forward,Front Beard"
9998,Female,1271,Mid,3,"Black Lipstick,Clown Eyes Green,Wild White Hair"
9999,Female,9233,Dark,2,"Mohawk,Nerd Glasses"


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   types         10000 non-null  object
 1   rarity        10000 non-null  int64 
 2   skin          10000 non-null  object
 3   total_traits  10000 non-null  int64 
 4   traits        9992 non-null   object
dtypes: int64(2), object(3)
memory usage: 390.8+ KB


In [8]:
# some missing values
df[df.traits.isnull()]

,types,rarity,skin,total_traits,traits
281,Male,7,Light,0,NaN
510,Male,9,Mid,0,NaN
641,Female,6,Light,0,NaN
741,Male,5,Dark,0,NaN
1050,Female,8,Mid,0,NaN
2204,Female,4,Dark,0,NaN
3307,Male,3,Albino,0,NaN
6487,Female,2,Albino,0,NaN


In [9]:
df.fillna('',inplace=True)

In [10]:
df1 = df.copy()

In [11]:
#examples of regex
text1 = 'Blonde Bob,Earring,Green Eye Shadow,Hidden Earring,,Hidden Eear'
text2 = 'Blonde Bob,Earring,Green Eye Shadow'

def extract_hidden_traits(text):
    match = re.findall(',Hidden\s(\w+)',text)
    if not match:
        #print("List is empty")
        return '' 
    else:
        return ','.join(match)

extract_hidden_traits(text2)
extract_hidden_traits(text1)

'Earring,Eear'

In [12]:
def get_number_hidden_traits(text):
    pattern = 'Hidden\s(\w+)'
    try:
        hidden = re.findall(pattern,text)
        return len(hidden)    
    except:
        return 0

In [13]:
df1['total_hidden_traits']=df1['traits'].apply(get_number_hidden_traits)

In [14]:
df1['hidden_traits']=df1['traits'].apply(extract_hidden_traits)

In [15]:
text1 = 'Blonde Bob,Earring,Green Eye Shadow,Hidden Earring,Hidden Eear'

def remove_hidden_traits(text):
    match = re.findall(',Hidden\s(\w+)',text)    
    if not match:
        #print("List is empty")
        return text 
    else:
        to_remove= ',Hidden '+',Hidden '.join(match)        
        clean = text.replace(to_remove,'')        
        return clean


remove_hidden_traits(text1)
    

'Blonde Bob,Earring,Green Eye Shadow'

In [16]:
df1['traits'] = df1['traits'].apply(remove_hidden_traits)

In [17]:
df1

,types,rarity,skin,total_traits,traits,total_hidden_traits,hidden_traits
0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow",1,Earring
1,Male,4352,Dark,2,"Mohawk,Smile",0,
2,Female,8090,Light,1,Wild Hair,0,
3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair",0,
4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair",0,
...,...,...,...,...,...,...,...
9995,Female,2466,Albino,2,"Purple Eye Shadow,Straight Hair Dark",0,
9996,Male,4323,Light,4,"Cigarette,Crazy Hair,Earring,Smile",0,
9997,Zombie,498,Zombie,2,"Cap Forward,Front Beard",0,
9998,Female,1271,Mid,3,"Black Lipstick,Clown Eyes Green,Wild White Hair",0,


In [39]:
df1[['types','rarity','skin','total_traits','total_hidden_traits']]

,types,rarity,skin,total_traits,total_hidden_traits
0,Female,2023,Mid,3,1
1,Male,4352,Dark,2,0
2,Female,8090,Light,1,0
3,Male,7896,Dark,3,0
4,Male,7332,Mid,4,0
...,...,...,...,...,...
9995,Female,2466,Albino,2,0
9996,Male,4323,Light,4,0
9997,Zombie,498,Zombie,2,0
9998,Female,1271,Mid,3,0


In [40]:
df1[['types','rarity','skin','total_traits','total_hidden_traits']].to_csv('punks.csv')

In [19]:
tipos = {'Alien':1,'Ape':2,'Zombie':3,'Male':4,'Female':5}

In [20]:
df['punk_type_id']= df['types'].map(tipos)

In [21]:
piel = {'Alien':1,'Ape':2,'Zombie':3,'Albino':4,'Dark':5,'Light':6,'Mid':7}

In [22]:
df['punk_skin_id']= df['skin'].map(piel)

df.to_csv('punks_atributes.csv')

In [23]:
df.head()

,types,rarity,skin,total_traits,traits,punk_type_id,punk_skin_id
0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow,Hidden Ear...",5,7
1,Male,4352,Dark,2,"Mohawk,Smile",4,5
2,Female,8090,Light,1,Wild Hair,5,6
3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair",4,5
4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair",4,7


In [28]:
mydb = mysql.connector.connect(
    host = 'localhost',    
    user = 'manel',
    password = 'itacademy',
    database="mydb")

mycursor = mydb.cursor()


In [33]:
sql_query = 'SHOW TABLES'
mycursor.execute(sql_query)

for x in mycursor:
  print(x) 

('punk',)
('punk_skin',)
('punk_type',)
('shops',)
('traits',)
('transactions',)


In [31]:
# executo una comanda de SQL language
sql_query = 'SELECT * FROM punk_skin;'
mycursor.execute(sql_query)
myresult = mycursor.fetchmany()
for x in myresult:
  print(x)

In [32]:
# executo una comanda de SQL language
sql_query = 'SELECT * FROM punk_skin'
mycursor.execute(sql_query)
myresult = mycursor.fetchall()
print(myresult)

[]
